In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib.pyplot as plt
from copy import copy

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.controllers_drake_2 import HybridModelPredictiveController

ImportError: No module named controllers_drake_2

In [ ]:
def get_2d_regular_polygon(n_factes):
    A = np.zeros((0, 2))
    b = np.zeros((0, 1))
    obj = Polyhedron(A, b)
    for i in range(n_facets):
        Ai = - np.array([[np.cos(2*i*np.pi/n_facets), np.sin(2*i*np.pi/n_facets)]])
        bi = np.ones((1, 1))
        obj.add_inequality(Ai, bi)
    return obj

In [ ]:
n_facets = 8
assert n_facets % 2 == 0
obj = get_2d_regular_polygon(n_factes)
obj.plot()
control_surfaces = [0, n_facets/2]

In [ ]:
x_max = 2.*np.ones((2,1))
x_min = - x_max
bounding_box = Polyhedron.from_bounds(x_min, x_max)
finger_free_spaces = []
finger_contact_spaces = []
for i in range(n_facets):
    facet = copy(obj)
    facet.add_equality(obj.A[i:i+1,:], obj.b[i,:])
    free_space = copy(bounding_box)
    free_space.add_inequality(-obj.A[i:i+1,:], -obj.b[i,:])
    contact_space = Polyhedron(obj.A[i:i+1,:], obj.b[i,:])
    for v in facet.vertices:
        if np.array([[v[1,0], -v[0,0]]]).dot(facet.center) < 0:
            free_space.add_inequality(np.array([[v[1,0], -v[0,0]]]), np.zeros((1,1)))
            contact_space.add_inequality(np.array([[v[1,0], -v[0,0]]]), np.zeros((1,1)))
        else:
            free_space.add_inequality(np.array([[-v[1,0], v[0,0]]]), np.zeros((1,1)))
            contact_space.add_inequality(np.array([[-v[1,0], v[0,0]]]), np.zeros((1,1)))
    finger_free_spaces.append(free_space)
    finger_contact_spaces.append(contact_space)

In [ ]:
import gurobipy as grb
prog = grb.Model()

In [ ]:
x = prog.addVars(2, lb=[-1.,-1.], name='x')
prog.update()
print x

In [ ]:
prog.setObjective(x[0]*x[0]+x[1]*x[1])
prog.addConstr(x[0] >= 1.)

In [ ]:
prog.setParam('Method', 0)
prog.optimize()

In [ ]:
x[0].VBasis = 0
x[1].VBasis = -1
prog.getConstrs()[0].CBasis = 0
prog.update()
print x[0].VBasis

In [ ]:
prog.getConstrs()[0].CBasis